In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df.drop(columns= ["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T15", "T29", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
classifications_to_replace = []
classif = pd.DataFrame(application_df["CLASSIFICATION"].value_counts())
all_cls = list(classif.index.values)
all_cls.remove("C1000")
all_cls.remove("C2000")
all_cls.remove("C1200")
all_cls.remove("C3000")
all_cls.remove("C2100")

# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = all_cls

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
new_app_df = pd.get_dummies(application_df)

In [6]:
# Split our preprocessed data into our features and target arrays
X = new_app_df.drop(columns="IS_SUCCESSFUL").values
y = new_app_df["IS_SUCCESSFUL"].values
print(X.shape)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

(34299, 43)


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer changed to have more units
nn.add(tf.keras.layers.Dense(units=130, activation="relu", input_dim=43))

# Second hidden layer changed to have more units
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Third hidden layer added
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer, activation changed
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 130)               5720      
                                                                 
 dense_1 (Dense)             (None, 60)                7860      
                                                                 
 dense_2 (Dense)             (None, 15)                915       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 14,511
Trainable params: 14,511
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model, using more epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 883us/step - loss: 0.6529 - accuracy: 0.7117
Epoch 2/200
804/804 [==============================] - 1s 879us/step - loss: 0.5736 - accuracy: 0.7263
Epoch 3/200
804/804 [==============================] - 1s 857us/step - loss: 0.6256 - accuracy: 0.7298
Epoch 4/200
804/804 [==============================] - 1s 861us/step - loss: 0.6310 - accuracy: 0.7305
Epoch 5/200
804/804 [==============================] - 1s 858us/step - loss: 0.6240 - accuracy: 0.7318
Epoch 6/200
804/804 [==============================] - 1s 854us/step - loss: 0.5531 - accuracy: 0.7323
Epoch 7/200
804/804 [==============================] - 1s 852us/step - loss: 0.5531 - accuracy: 0.7329
Epoch 8/200
804/804 [==============================] - 1s 853us/step - loss: 0.5847 - accuracy: 0.7324
Epoch 9/200
804/804 [==============================] - 1s 863us/step - loss: 0.5627 - accuracy: 0.7345
Epoch 10/200
804/804 [==============================] - 1s 978us/step - l

804/804 [==============================] - 1s 857us/step - loss: 0.5422 - accuracy: 0.7400
Epoch 80/200
804/804 [==============================] - 1s 863us/step - loss: 0.5433 - accuracy: 0.7398
Epoch 81/200
804/804 [==============================] - 1s 870us/step - loss: 0.5438 - accuracy: 0.7397
Epoch 82/200
804/804 [==============================] - 1s 838us/step - loss: 0.5437 - accuracy: 0.7399
Epoch 83/200
804/804 [==============================] - 1s 844us/step - loss: 0.5421 - accuracy: 0.7411
Epoch 84/200
804/804 [==============================] - 1s 837us/step - loss: 0.5407 - accuracy: 0.7414
Epoch 85/200
804/804 [==============================] - 1s 849us/step - loss: 0.5403 - accuracy: 0.7397
Epoch 86/200
804/804 [==============================] - 1s 856us/step - loss: 0.5379 - accuracy: 0.7401
Epoch 87/200
804/804 [==============================] - 1s 879us/step - loss: 0.5391 - accuracy: 0.7399
Epoch 88/200
804/804 [==============================] - 1s 839us/step - loss:

804/804 [==============================] - 1s 841us/step - loss: 0.5449 - accuracy: 0.7417
Epoch 158/200
804/804 [==============================] - 1s 846us/step - loss: 0.5427 - accuracy: 0.7405
Epoch 159/200
804/804 [==============================] - 1s 842us/step - loss: 0.5392 - accuracy: 0.7424
Epoch 160/200
804/804 [==============================] - 1s 848us/step - loss: 0.5397 - accuracy: 0.7419
Epoch 161/200
804/804 [==============================] - 1s 844us/step - loss: 0.5393 - accuracy: 0.7414
Epoch 162/200
804/804 [==============================] - 1s 846us/step - loss: 0.5388 - accuracy: 0.7412
Epoch 163/200
804/804 [==============================] - 1s 918us/step - loss: 0.5403 - accuracy: 0.7412
Epoch 164/200
804/804 [==============================] - 1s 898us/step - loss: 0.5396 - accuracy: 0.7427
Epoch 165/200
804/804 [==============================] - 1s 844us/step - loss: 0.5402 - accuracy: 0.7411
Epoch 166/200
804/804 [==============================] - 1s 849us/ste

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6391 - accuracy: 0.7257 - 253ms/epoch - 944us/step
Loss: 0.6390597224235535, Accuracy: 0.7257142663002014


This optimization attempt made the accuracy worse than the original model.

In [15]:
# Implementing a keras model tuner to improve accuracy
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int("first_units", min_value= 1, max_value= 150, step= 2),
                                      activation= activation, input_dim= 43))
    
    for i in range(hp.Int("num_layers", 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i), min_value= 1, max_value= 150, step=2),
                                           activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return nn_model

In [16]:
import keras_tuner as kt

tuner = kt.Hyperband(create_model, objective="val_accuracy", max_epochs = 100, hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [17]:
tuner.search(X_train_scaled, y_train, epochs= 100, validation_data=(X_test_scaled, y_test))

Trial 508 Complete [00h 01m 42s]
val_accuracy: 0.7258309125900269

Best val_accuracy So Far: 0.7281632423400879
Total elapsed time: 01h 42m 15s
INFO:tensorflow:Oracle triggered exit


In [50]:
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 135, 'num_layers': 2, 'units_0': 69, 'units_1': 11, 'units_2': 119, 'units_3': 81, 'units_4': 89, 'units_5': 71, 'tuner/epochs': 12, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [51]:
tuner_model = tuner.get_best_models(1)
for model in tuner_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5527 - accuracy: 0.7282 - 270ms/epoch - 1ms/step
Loss: 0.55266934633255, Accuracy: 0.7281632423400879


In [57]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Some tuner values are pulled into a new model
tuned_nn = tf.keras.models.Sequential()

# First hidden layer
tuned_nn.add(tf.keras.layers.Dense(units=135, activation="relu", input_dim=43))

# Second hidden layer
tuned_nn.add(tf.keras.layers.Dense(units=69, activation="relu"))

# Third hidden layer
tuned_nn.add(tf.keras.layers.Dense(units=11, activation="relu"))

# Output layer, activation changed
tuned_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
tuned_nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 135)               5940      
                                                                 
 dense_9 (Dense)             (None, 69)                9384      
                                                                 
 dense_10 (Dense)            (None, 11)                770       
                                                                 
 dense_11 (Dense)            (None, 1)                 12        
                                                                 
Total params: 16,106
Trainable params: 16,106
Non-trainable params: 0
_________________________________________________________________


In [58]:
tuned_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
tuned_fit_model = tuned_nn.fit(X_train_scaled, y_train, epochs=12)

Epoch 1/12
804/804 [==============================] - 1s 841us/step - loss: 0.5686 - accuracy: 0.7233
Epoch 2/12
804/804 [==============================] - 1s 854us/step - loss: 0.5536 - accuracy: 0.7304
Epoch 3/12
804/804 [==============================] - 1s 893us/step - loss: 0.5516 - accuracy: 0.7318
Epoch 4/12
804/804 [==============================] - 1s 849us/step - loss: 0.5492 - accuracy: 0.7346
Epoch 5/12
804/804 [==============================] - 1s 872us/step - loss: 0.5486 - accuracy: 0.7346
Epoch 6/12
804/804 [==============================] - 1s 885us/step - loss: 0.5480 - accuracy: 0.7345
Epoch 7/12
804/804 [==============================] - 1s 844us/step - loss: 0.5478 - accuracy: 0.7348
Epoch 8/12
804/804 [==============================] - 1s 844us/step - loss: 0.5461 - accuracy: 0.7362
Epoch 9/12
804/804 [==============================] - 1s 846us/step - loss: 0.5458 - accuracy: 0.7347
Epoch 10/12
804/804 [==============================] - 1s 859us/step - loss: 0.545

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = tuned_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7261 - 229ms/epoch - 854us/step
Loss: 0.555141031742096, Accuracy: 0.726064145565033


Based on these 3 attempts, the best attempt was the tuner model, which will be saved into a HDF5 file.

In [62]:
for model in tuner_model:
    model.save("AlphabetSoupCharity_Optimization.h5")